In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score
import random

In [7]:

#Metadaten einlesen
df = pd.read_csv("NIHMS841832-supplement-1.csv", sep=',')
    
#Ergebnisse des Feature Tables einlesen
feature = pd.read_csv('feature_table.txt', sep='\t').T
feature = feature[1:][:-1]

feature

,0,1,2,3,4,5,6,7,8,9,...,3101,3102,3103,3104,3105,3106,3107,3108,3109,3110
1629.SubjectIBD335,34292,20670,18413,9981,7071,6881,5411,5335,5289,4741,...,0,0,0,0,0,0,0,0,0,0
1629.SubjectIBD643,15243,64328,0,0,0,4,4507,3216,15630,199,...,0,0,0,0,0,0,0,0,0,0
1629.SubjectIBD539,22182,21589,0,1365,0,11501,33619,3638,5053,0,...,0,0,0,0,0,0,0,0,0,0
1629.SubjectIBD078,0,805,0,0,0,4,330,2305,0,8,...,0,0,0,0,0,0,0,0,0,0
1629.SubjectIBD671,0,19734,0,0,0,0,215,0,0,699,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629.SubjectIBD421,5154,12101,1572,62,190,1448,6316,2205,1885,0,...,0,0,0,0,0,0,0,0,0,0
1629.SubjectIBD202,14565,24920,3543,0,0,0,63,0,37768,48660,...,0,0,0,0,0,0,0,0,0,0
1629.SubjectIBD544,32,52,31,0,0,2,18543,0,3,45,...,0,0,0,0,0,0,0,0,946,31
1629.SubjectIBD422,5718,18420,9534,0,0,0,4791,2770,0,588,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#Gesunde Kontrollgruppe
HC = df[df.ibd_subtype.eq("HC")]

y = []
for row in feature.index:
    if any(True for val in HC['sample_name'] if val == row):
        y.append(1)
    else:
        y.append(0)

X = feature.iloc[:, :].values  


In [9]:
#Splitt nach Patient. Alle Zeitpunkte eines Patienten entweder in Train oder Test.
def split_function(tSize, random_state, table, metadata, hc_group):
    patientSamples = {}
    liste = []

    for row in metadata.index:
        liste.append(metadata['patientnumber'][row])

    menge = set(liste)  

    for e in menge:
        newPatient = metadata[metadata.patientnumber.eq(e)]
        patientSamples[e] = list(newPatient['sample_name'])

    shuffleListe = list(menge)
    random.Random(random_state).shuffle(shuffleListe)

    #Split. 137 PatientenNummern
    trainSize = int(np.round(tSize * 137, 0))
    testSize = 137 - trainSize

    X_trainEntry = shuffleListe[0:trainSize]
    X_testEntry = shuffleListe[trainSize:138]

    X_train = []
    y_train = []
    for i in X_trainEntry:
        try:
            value = patientSamples[i]
            for sample_name in value:
                for row in table.index:
                    if(row == sample_name):
                        temp = table.loc[[row]].values[0]
                        X_train.append(temp)
                        if any(True for val in hc_group['sample_name'] if val == row):
                            y_train.append(1)
                        else:
                            y_train.append(0)
        except KeyError as e:
            fehler += 1
            print('I got a KeyError - reason "%s"' % str(e))


    X_test = []   
    y_test = []
    for i in X_testEntry:
        try:
            value = patientSamples[i]
            for sample_name in value:
                for row in table.index:
                    if(row == sample_name):
                        temp = table.loc[[row]].values[0]
                        X_test.append(temp)
                        if any(True for val in hc_group['sample_name'] if val == row):
                            y_test.append(1)
                        else:
                            y_test.append(0)
        except KeyError as e:
            fehler += 1
            print('I got a KeyError - reason "%s"' % str(e))

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    return X_train, X_test, y_train, y_test

In [10]:
f1Sum = 0.0
n = 100
for i in range(0, n):
    #Split Test und Trainingsdaten
    X_train, X_test, y_train, y_test = split_function(tSize=0.7, random_state=i, table=feature, metadata=df, hc_group=HC)
    trainSize = len(X_train)/680
    testSize =len(X_test)/680 
    
    #Train
    classifier = svm.SVC(kernel='linear')
    classifier.fit(X_train, y_train)
    
    #Test
    y_pred = classifier.predict(X_test)
    predict_train = classifier.predict(X_train)
    f1 = f1_score(y_test, y_pred, average='macro')
    print(str(i) + ": " + "f1_score: " + str(f1) + "    " + str(np.round(trainSize, 2)) +"/"+ str(np.round(testSize, 2)) + " Split")
    f1Sum += f1
f1 = f1Sum/n
print("f1-score = "+str(f1))

0: f1_score: 0.5667975667975668    0.72/0.29 Split
1: f1_score: 0.6041079531884403    0.67/0.33 Split
2: f1_score: 0.6378947368421053    0.69/0.32 Split
3: f1_score: 0.47143774069319644    0.73/0.27 Split
4: f1_score: 0.5743810108450992    0.7/0.31 Split
5: f1_score: 0.5108582110066524    0.68/0.32 Split
6: f1_score: 0.5453393321082315    0.7/0.3 Split
7: f1_score: 0.5490054249547921    0.69/0.32 Split
8: f1_score: 0.5963711248399844    0.71/0.29 Split
9: f1_score: 0.6151828847481021    0.7/0.3 Split
10: f1_score: 0.4578005115089514    0.69/0.31 Split
11: f1_score: 0.45618556701030927    0.69/0.31 Split
12: f1_score: 0.4441176470588235    0.72/0.28 Split
13: f1_score: 0.45454545454545453    0.73/0.27 Split
14: f1_score: 0.5336927223719676    0.75/0.25 Split
15: f1_score: 0.6121739130434782    0.67/0.33 Split
16: f1_score: 0.7010207237859574    0.74/0.26 Split
17: f1_score: 0.6785798816568047    0.72/0.29 Split
18: f1_score: 0.4651810584958217    0.72/0.28 Split
19: f1_score: 0.53756723